In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import glob
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [3]:
PATH_TRAIN = "B:\\CV Drone Detection\\scaled\\train"
PATH_TEST = "B:\\CV Drone Detection\\scaled\\test"
PATH_VALID = "B:\\CV Drone Detection\\scaled\\valid"

In [ ]:


transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),   # різні масштаби об’єкта
    transforms.RandomHorizontalFlip(),                     # віддзеркалення
    transforms.RandomVerticalFlip(),                       
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomRotation(20),                         
    transforms.RandomAffine(degrees=0, translate=(0.1,0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.Resize((128)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.2)),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])


trainset = torchvision.datasets.ImageFolder(root=PATH_TRAIN, transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root=PATH_TEST, transform=transform)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = trainset.classes
print("Classes:", classes)


Classes: ['drone', 'notAdrone']


In [5]:
from collections import Counter
print("Train:", Counter(trainset.targets))
print("Valid:", Counter(testset.targets))

Train: Counter({0: 8740, 1: 5563})
Valid: Counter({1: 650, 0: 339})


In [6]:
# import torch.nn as nn

# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(5, 5)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, len(classes))

#     def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = x.view(x.size(0), -1)
#         x = nn.functional.relu(self.fc1(x))
#         x = nn.functional.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# net = Net()

In [39]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, padding=1)

        with torch.no_grad():
            dummy = torch.zeros(1, 3, 128, 128)
            out = self.pool(F.relu(self.conv1(dummy)))
            out = self.pool(F.relu(self.conv2(out)))
            n_features = out.view(1, -1).size(1)

        self.fc1 = nn.Linear(n_features, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 30 * 30)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net(num_classes=len(classes))

In [40]:
criterion = nn.CrossEntropyLoss()


optimizer = optim.Adam(net.parameters(), lr=1e-4)

# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [41]:
for epoch in range(2):  

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999: 
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.354
[2,  2000] loss: 0.199
Finished Training


In [42]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the  test images: %d %%' % (
    100 * correct / total), correct, total)

Accuracy of the network on the  test images: 93 % 929 989


In [43]:
from sklearn.metrics import classification_report

y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

print(classification_report(y_true, y_pred, target_names=classes))

              precision    recall  f1-score   support

       drone       0.88      0.91      0.90       339
   notAdrone       0.95      0.94      0.95       650

    accuracy                           0.93       989
   macro avg       0.92      0.93      0.92       989
weighted avg       0.93      0.93      0.93       989



In [44]:
image_paths = {
    "notAdrone": glob.glob(r'B:\CV Drone Detection\scaled\valid\notAdrone\*.jpg'),
    "drone": glob.glob(r'B:\CV Drone Detection\scaled\valid\drone\*.jpg')
}

In [45]:
val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

class_to_idx = trainset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

def multi_glob(pattern_base):
    return (
        glob.glob(pattern_base + "*.jpg") +
        glob.glob(pattern_base + "*.jpeg") +
        glob.glob(pattern_base + "*.png")
    )

image_paths = {
    "drone":     multi_glob(r"B:\CV Drone Detection\scaled\valid\drone\\"),
    "notAdrone": multi_glob(r"B:\CV Drone Detection\scaled\valid\notAdrone\\"),
}

device = next(net.parameters()).device
def model_testing(image_paths_dict, net, transform):
    net.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for class_name, paths in image_paths_dict.items():
            class_idx = class_to_idx[class_name]
            for p in paths:
                img = Image.open(p).convert("RGB")
                inp = transform(img).unsqueeze(0).to(device)
                out = net(inp)
                pred = out.argmax(dim=1).item()
                y_true.append(class_idx)
                y_pred.append(pred)
    return y_true, y_pred

y_true, y_pred = model_testing(image_paths, net, val_transform)
target_names = [idx_to_class[i] for i in range(len(idx_to_class))]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       drone       1.00      0.82      0.90       721
   notAdrone       0.72      1.00      0.83       325

    accuracy                           0.88      1046
   macro avg       0.86      0.91      0.87      1046
weighted avg       0.91      0.88      0.88      1046

